In [2]:
import pefile
import os

In [6]:
def getNFiles(root="/", n_files=500):
    stored_files = []
    for dirpath, dirs, files in os.walk(root, topdown=False):
        for filename in files:
            if filename.endswith(".exe"):
                stored_files.append(dirpath + "/" + filename)
            
            if len(stored_files) >= n_files: return stored_files
    
    return stored_files

files = getNFiles(n_files=500)
print(files)

['/usr/lib/python3.13/site-packages/pip/_vendor/distlib/w64.exe', '/usr/lib/python3.13/site-packages/pip/_vendor/distlib/w64-arm.exe', '/usr/lib/python3.13/site-packages/pip/_vendor/distlib/t32.exe', '/usr/lib/python3.13/site-packages/pip/_vendor/distlib/t64.exe', '/usr/lib/python3.13/site-packages/pip/_vendor/distlib/t64-arm.exe', '/usr/lib/python3.13/site-packages/pip/_vendor/distlib/w32.exe', '/opt/miniconda3/pkgs/setuptools-72.1.0-py312h06a4308_0/lib/python3.12/site-packages/setuptools/_distutils/command/wininst-14.0.exe', '/opt/miniconda3/pkgs/setuptools-72.1.0-py312h06a4308_0/lib/python3.12/site-packages/setuptools/_distutils/command/wininst-9.0-amd64.exe', '/opt/miniconda3/pkgs/setuptools-72.1.0-py312h06a4308_0/lib/python3.12/site-packages/setuptools/_distutils/command/wininst-14.0-amd64.exe', '/opt/miniconda3/pkgs/setuptools-72.1.0-py312h06a4308_0/lib/python3.12/site-packages/setuptools/_distutils/command/wininst-10.0-amd64.exe', '/opt/miniconda3/pkgs/setuptools-72.1.0-py312h06

# **Extracting file data**

In [4]:
pe = pefile.PE(files[0])
pe.full_load()

## 1.) Imported Symbols

In [5]:
def getImports(pe: pefile.PE):
    imports = []
    for entry in pe.DIRECTORY_ENTRY_IMPORT:
        for imp in entry.imports:
            imports.append(imp.name.decode("utf-8"))
    
    return imports

### 1.1) Get all imports and weight them

In [ ]:
from math import log2

# Returns all imports in the training dataset
def getGlobalImpVocab(training_files: list):
    import_vocab = set()
    for file_path in training_files:
        try: 
            pe = pefile.PE(file_path)
            #pe.full_load()

            imports = getImports(pe)
            for imp in imports:
                import_vocab.add(imp)
        
            pe.close()  # free memory

        except Exception as e:
            print("Failed to read file: ", file_path, e)

    return import_vocab


def getTrainingIDF(global_import_vocab, training_files: list):
    import_count = {imp:0 for imp in global_import_vocab}
    IDF = {}

    for file_path in training_files:
        try:
            pe = pefile.PE(file_path)
            pe.full_load()

            imports = getImports(pe)
            for imp in imports:
                import_count[imp] = import_count.get(imp, 0) + 1
    
                
        except Exception as e:
            print("Failed with file: ", file_path)
            continue

    N = len(training_files)
    IDF = {imp:log2(N/import_count[imp]) for imp in import_count.keys()}
    return IDF


def get_TF_IDF(global_import_vocab, training_IDF, new_file):
    TF_IDF = {imp:0 for imp in global_import_vocab}

    try:
        pe = pefile.PE(new_file)
        pe.full_load()

        imports = getImports(pe)
        for imp in imports:
            TF_IDF[imp] += training_IDF[imp]


    except Exception as e:
        print("Failed with file: ", e)
        return None

    return TF_IDF

global_import_vocab = getGlobalImpVocab(files)
print("global_import_vocab: ", global_import_vocab)

IDF = getTrainingIDF(global_import_vocab, files)
print(IDF)

TF_IDF = get_TF_IDF(global_import_vocab, IDF, files[0])
print(TF_IDF)


Failed to read file:  /home/jocke/Mathematica/SystemFiles/Links/TinkerForgeWeatherStationTools/Binaries/Linux-x86-64/TinkerForgeWeatherStationTools.exe 'DOS Header magic not found.'
Failed to read file:  /home/jocke/Mathematica/SystemFiles/Activation/Linux-x86-64/reqpwd.exe 'DOS Header magic not found.'
Failed to read file:  /home/jocke/Mathematica/SystemFiles/Converters/Binaries/Linux-x86-64/MatrixMarket.exe 'DOS Header magic not found.'
Failed to read file:  /home/jocke/Mathematica/SystemFiles/Converters/Binaries/Linux-x86-64/GZIP.exe 'DOS Header magic not found.'
Failed to read file:  /home/jocke/Mathematica/SystemFiles/Converters/Binaries/Linux-x86-64/GDAL.exe 'DOS Header magic not found.'
Failed to read file:  /home/jocke/Mathematica/SystemFiles/Converters/Binaries/Linux-x86-64/HarwellBoeing.exe 'DOS Header magic not found.'
Failed to read file:  /home/jocke/Mathematica/SystemFiles/Converters/Binaries/Linux-x86-64/NBImport.exe 'DOS Header magic not found.'
Failed to read file:  /h

# Autoencoder